# Week 9: Local Motion Planning

## Module III: Planning & Decision Making

### Topics Covered

- Configuration Space and Collision Detection
- Sampling-based Planners (RRT, RRT*)
- Search-based Planners (State Lattice, Hybrid A*)
- Path Smoothing and Optimization
- Real-time Motion Planning

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Understand configuration space and its role in motion planning
2. Implement RRT (Rapidly-Exploring Random Trees) for probabilistic planning
3. Extend RRT to RRT* for asymptotically optimal paths
4. Build state lattice planners with kinematically-feasible motion primitives
5. Perform collision detection for various obstacle shapes
6. Apply path smoothing techniques to improve trajectory quality
7. Compare sampling-based vs search-based planning approaches

---

## Setup

Import required libraries

In [ ]:
# Google Colab optimizations
import gc
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend for lower memory
import matplotlib.pyplot as plt

# Reduce matplotlib memory usage
plt.rcParams['figure.max_open_warning'] = 0
plt.rcParams['agg.path.chunksize'] = 10000

# Memory optimization: Use %matplotlib inline instead of widget for lower RAM usage
# If experiencing memory issues, restart kernel between major cells

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle, Polygon
from matplotlib.collections import LineCollection
import time

# Set random seed for reproducibility
np.random.seed(42)

## 1. Motion Planning Fundamentals

Local motion planning generates **collision-free trajectories** from the current position to a nearby goal, accounting for obstacles and vehicle dynamics.

### Planning Hierarchy

1. **Mission Planning**: Route on road network (A*, Dijkstra)
2. **Behavior Planning**: High-level decisions (FSM, behavior trees)
3. **Local Motion Planning**: Collision-free trajectory generation ← **This week**
4. **Control**: Execute trajectory (PID, MPC)

### Configuration Space (C-space)

**Definition:** The space of all possible robot configurations.

For a 2D vehicle:
- **Work Space**: Physical (x, y) space with obstacles
- **Configuration Space**: (x, y, θ) - position + heading
- **State Space**: (x, y, θ, v) - includes velocity

**C-space Obstacle:** Set of configurations where robot collides with obstacles.

### Motion Planning Problem

**Given:**
- Start configuration $q_{start}$
- Goal configuration $q_{goal}$
- Obstacle set $\mathcal{O}$ in C-space

**Find:**
- Path $\pi: [0, 1] \rightarrow \mathcal{C}_{free}$
- Where $\pi(0) = q_{start}$, $\pi(1) = q_{goal}$
- And $\pi(s) \notin \mathcal{O}$ for all $s \in [0, 1]$

**Objective:** Minimize cost (e.g., path length, time, energy)

### Planning Approaches

| Approach | Method | Completeness | Optimality |
|----------|--------|--------------|------------|
| **Sampling-based** | RRT, RRT* | Probabilistic | RRT*: Asymptotic |
| **Search-based** | A*, State Lattice | Resolution | Yes (discretization) |
| **Optimization** | Trajectory optimization | Local | Local minimum |

### Challenges in Autonomous Driving

1. **High-dimensional state space**: (x, y, θ, v, a)
2. **Nonholonomic constraints**: Can't move sideways
3. **Dynamic obstacles**: Other vehicles, pedestrians
4. **Real-time requirements**: < 100ms planning cycles
5. **Safety**: Must guarantee collision avoidance

## 2. Rapidly-Exploring Random Trees (RRT)

**RRT** is a sampling-based algorithm that builds a tree of collision-free paths by random exploration.

### Algorithm Overview

**Core Idea**: Incrementally grow a tree from $q_{start}$ by randomly sampling the configuration space and connecting to the nearest existing node.

**Algorithm Steps**:

1. **Initialize**: Tree $T$ with root at $q_{start}$
2. **Loop** for $N$ iterations:
   - **Sample**: Random configuration $q_{rand}$
   - **Nearest**: Find nearest node $q_{near}$ in tree to $q_{rand}$
   - **Steer**: Generate new node $q_{new}$ from $q_{near}$ towards $q_{rand}$ (step size $\delta$)
   - **Collision Check**: If path from $q_{near}$ to $q_{new}$ is collision-free:
     - **Add**: $q_{new}$ to tree with edge from $q_{near}$
   - **Goal Check**: If $q_{new}$ is close enough to $q_{goal}$, done!

**Pseudocode**:
```
RRT(q_start, q_goal, n_iterations):
    T.init(q_start)
    
    for i = 1 to n_iterations:
        q_rand = Sample()
        q_near = Nearest(T, q_rand)
        q_new = Steer(q_near, q_rand, step_size)
        
        if CollisionFree(q_near, q_new):
            T.add_vertex(q_new)
            T.add_edge(q_near, q_new)
            
            if Distance(q_new, q_goal) < threshold:
                return Path(T, q_start, q_new)
    
    return FAILURE
```

---

### Key Components

#### **Sampling**

**Uniform Random Sampling**: 
$$q_{rand} \sim \mathcal{U}(\mathcal{C}_{space})$$

**Goal Biasing** (common enhancement):
- With probability $p_{goal}$ (e.g., 0.05), sample $q_{goal}$
- Speeds convergence by occasionally pulling tree towards goal

#### **Nearest Neighbor**

Find $q_{near} = \arg\min_{q \in T} d(q, q_{rand})$

**Distance Metrics**:
- Euclidean: $d(q_1, q_2) = \sqrt{(x_2-x_1)^2 + (y_2-y_1)^2}$
- For configurations with heading: $d = \sqrt{(x_2-x_1)^2 + (y_2-y_1)^2 + w(\theta_2-\theta_1)^2}$

**Data Structures**:
- Naive: Linear search $O(n)$
- **KD-Tree**: $O(\log n)$ average case

#### **Steering**

Generate $q_{new}$ at most $\delta$ distance from $q_{near}$ towards $q_{rand}$:

$$q_{new} = q_{near} + \min\left(\delta, d(q_{near}, q_{rand})\right) \cdot \frac{q_{rand} - q_{near}}{d(q_{near}, q_{rand})}$$

**Purpose**: 
- Limits tree growth per iteration
- Ensures dense coverage

#### **Collision Checking**

Check if path from $q_{near}$ to $q_{new}$ intersects any obstacle.

**Methods**:
- **Discretization**: Sample points along path
- **Line-obstacle intersection**: Geometric tests

---

### Properties

**Probabilistic Completeness**: 
- If a solution exists, RRT will find it with probability approaching 1 as iterations → ∞

**NOT Optimal**:
- First solution found is typically far from optimal
- No guarantee on path quality

**Space Exploration**:
- **Voronoi Bias**: Tends to explore large empty regions
- Good for finding solutions quickly
- Not focused on optimization

---

### RRT Variants

**RRT-Connect** (Bidirectional):
- Grow two trees: one from start, one from goal
- Try to connect them
- **2-4× faster** than single-tree RRT

**Goal-Biased RRT**:
- Sample goal with probability $p_{goal}$ (e.g., 5%)
- Faster convergence in simple environments

**Kinodynamic RRT**:
- Respects dynamics (acceleration limits, etc.)
- Uses control inputs instead of straight-line steering

---

### Limitations

1. **No optimality**: Path quality can be poor
2. **Parameter tuning**: Step size $\delta$ affects performance
3. **High dimensions**: Becomes slower in high-D spaces
4. **No smoothness**: Paths have sharp turns

**Solution**: Use RRT* for optimality

In [ ]:
# Complete RRT Implementation

class RRTNode:
    """Node in RRT tree."""
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.parent = None
        self.cost = 0.0  # Cost from start (for RRT*)
    
    def __repr__(self):
        return f"Node({self.x:.2f}, {self.y:.2f})"


class Environment:
    """2D environment with rectangular obstacles."""
    
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.obstacles = []  # List of (x, y, width, height)
    
    def add_obstacle(self, x, y, width, height):
        """Add rectangular obstacle."""
        self.obstacles.append((x, y, width, height))
    
    def is_collision(self, x, y, margin=0.5):
        """Check if point (x, y) collides with any obstacle."""
        # Check bounds
        if x < 0 or x > self.width or y < 0 or y > self.height:
            return True
        
        # Check obstacles
        for ox, oy, ow, oh in self.obstacles:
            if (ox - margin <= x <= ox + ow + margin and
                oy - margin <= y <= oy + oh + margin):
                return True
        return False
    
    def is_path_collision_free(self, x1, y1, x2, y2, resolution=0.5):
        """Check if line segment from (x1,y1) to (x2,y2) is collision-free."""
        distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        n_points = int(distance / resolution) + 1
        
        for i in range(n_points + 1):
            t = i / n_points if n_points > 0 else 0
            x = x1 + t * (x2 - x1)
            y = y1 + t * (y2 - y1)
            
            if self.is_collision(x, y):
                return False
        return True


class RRT:
    """RRT (Rapidly-Exploring Random Trees) planner."""
    
    def __init__(self, start, goal, environment, 
                 max_iterations=300, step_size=2.0, goal_sample_rate=0.05):
        """
        Args:
            start: (x, y) start position
            goal: (x, y) goal position
            environment: Environment object
            max_iterations: Maximum planning iterations
            step_size: Maximum distance to extend tree
            goal_sample_rate: Probability of sampling goal
        """
        self.start = RRTNode(start[0], start[1])
        self.goal = RRTNode(goal[0], goal[1])
        self.env = environment
        self.max_iterations = max_iterations
        self.step_size = step_size
        self.goal_sample_rate = goal_sample_rate
        
        self.nodes = [self.start]
        self.goal_threshold = 2.0
        
    def sample(self):
        """Sample random point in configuration space."""
        if np.random.rand() < self.goal_sample_rate:
            # Goal biasing
            return self.goal.x, self.goal.y
        else:
            # Uniform random
            x = np.random.uniform(0, self.env.width)
            y = np.random.uniform(0, self.env.height)
            return x, y
    
    def nearest(self, x, y):
        """Find nearest node in tree to (x, y)."""
        min_dist = float('inf')
        nearest_node = None
        
        for node in self.nodes:
            dist = np.sqrt((node.x - x)**2 + (node.y - y)**2)
            if dist < min_dist:
                min_dist = dist
                nearest_node = node
        
        return nearest_node
    
    def steer(self, from_node, to_x, to_y):
        """Create new node from from_node towards (to_x, to_y)."""
        # Direction vector
        dx = to_x - from_node.x
        dy = to_y - from_node.y
        distance = np.sqrt(dx**2 + dy**2)
        
        # Limit distance to step_size
        if distance > self.step_size:
            dx = dx / distance * self.step_size
            dy = dy / distance * self.step_size
        
        new_node = RRTNode(from_node.x + dx, from_node.y + dy)
        new_node.parent = from_node
        new_node.cost = from_node.cost + np.sqrt(dx**2 + dy**2)
        
        return new_node
    
    def plan(self):
        """Run RRT algorithm."""
        for i in range(self.max_iterations):
            # Sample
            x_rand, y_rand = self.sample()
            
            # Nearest
            nearest_node = self.nearest(x_rand, y_rand)
            
            # Steer
            new_node = self.steer(nearest_node, x_rand, y_rand)
            
            # Collision check
            if self.env.is_path_collision_free(
                nearest_node.x, nearest_node.y,
                new_node.x, new_node.y
            ):
                self.nodes.append(new_node)
                
                # Check if reached goal
                dist_to_goal = np.sqrt((new_node.x - self.goal.x)**2 + 
                                      (new_node.y - self.goal.y)**2)
                
                if dist_to_goal < self.goal_threshold:
                    # Connect to goal
                    if self.env.is_path_collision_free(
                        new_node.x, new_node.y,
                        self.goal.x, self.goal.y
                    ):
                        self.goal.parent = new_node
                        self.goal.cost = new_node.cost + dist_to_goal
                        self.nodes.append(self.goal)
                        return self.extract_path(), i + 1
        
        # Failed to find path
        return None, self.max_iterations
    
    def extract_path(self):
        """Extract path from start to goal."""
        if self.goal not in self.nodes:
            return None
        
        path = []
        current = self.goal
        
        while current is not None:
            path.append((current.x, current.y))
            current = current.parent
        
        path.reverse()
        return path


# Create environment
env = Environment(width=50, height=50)

# Add obstacles
env.add_obstacle(10, 10, 5, 15)
env.add_obstacle(25, 0, 5, 20)
env.add_obstacle(25, 25, 5, 15)
env.add_obstacle(35, 15, 10, 5)
env.add_obstacle(15, 30, 15, 5)

# Set start and goal
start = (5, 5)
goal = (45, 45)

# Run RRT
np.random.seed(42)
rrt = RRT(start, goal, env, max_iterations=300, step_size=2.0)
path, iterations = rrt.plan()

# Simple visualization - path only
fig, ax = plt.subplots(1, 1, figsize=(10, 8))

# Draw obstacles
for ox, oy, ow, oh in env.obstacles:
    rect = Rectangle((ox, oy), ow, oh, facecolor='gray', edgecolor='black', linewidth=2)
    ax.add_patch(rect)

# Draw path
if path:
    path_x, path_y = zip(*path)
    ax.plot(path_x, path_y, 'b-', linewidth=3, label='RRT Path', zorder=5)
    ax.scatter(path_x, path_y, c='blue', s=50, zorder=6)

    path_length = sum(np.sqrt((path[i+1][0]-path[i][0])**2+(path[i+1][1]-path[i][1])**2)
                     for i in range(len(path)-1))
    title = f'RRT Path (Colab Optimized)\nLength: {path_length:.2f}, Nodes: {len(rrt.nodes)}'
else:
    title = 'RRT: Path Not Found'

# Start and goal
ax.scatter([start[0]], [start[1]], c='green', s=300, marker='o',
          edgecolors='black', linewidths=2, label='Start', zorder=10)
ax.scatter([goal[0]], [goal[1]], c='red', s=300, marker='*',
          edgecolors='black', linewidths=2, label='Goal', zorder=10)

ax.set_xlim([0, env.width])
ax.set_ylim([0, env.height])
ax.set_xlabel('X', fontsize=12)
ax.set_ylabel('Y', fontsize=12)
ax.set_title(title, fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_aspect('equal')

plt.tight_layout()
plt.show()
plt.close(fig)

# Results
print("="*70)
print("RRT RESULTS")
print("="*70)
print(f"Iterations: {iterations}, Nodes: {len(rrt.nodes)}")
if path:
    print(f"Path length: {path_length:.2f}, Waypoints: {len(path)}")
print("="*70)

# Cleanup
del rrt, path
gc.collect()


## 3. RRT* - Optimal RRT

RRT* extends RRT with **rewiring** to achieve asymptotically optimal paths.

### Key Differences from RRT

| Aspect | RRT | RRT* |
|--------|-----|------|
| **Path Quality** | First solution (suboptimal) | Converges to optimal |
| **Rewiring** | No | Yes |
| **Nearest** | 1 nearest node | Nodes within radius |
| **Complexity** | $O(n)$ | $O(n \log n)$ |
| **Time** | Faster for first solution | Better final solution |

### Additional Steps

After adding $q_{new}$ to tree:

1. **Choose Best Parent**: Among nearby nodes, select parent that minimizes cost to $q_{new}$
2. **Rewire Tree**: For each nearby node, check if routing through $q_{new}$ reduces cost

### Algorithm Enhancement

```python
# After collision-free edge is found:
1. near_nodes = Near(T, q_new, radius)
2. min_cost_parent = ChooseBestParent(near_nodes, q_new)
3. T.add_vertex(q_new)
4. T.add_edge(min_cost_parent, q_new)
5. Rewire(T, near_nodes, q_new)  # Update tree structure
```

### Shrinking Radius

For asymptotic optimality, use:
$$r_n = \min\left\{ \gamma \left(\frac{\log n}{n}\right)^{1/d}, \delta \right\}$$

Where:
- $n$ = number of nodes
- $d$ = dimension of C-space
- $\gamma$ = constant (typically 10-30)
- $\delta$ = max step size

### Complexity

- **RRT**: $O(n)$ per iteration
- **RRT***: $O(n \log n)$ per iteration (due to near neighbor search)

**Trade-off**: Slower initial solution, better final path quality

In [ ]:
# NOTE: Run cell 5 first to define the RRT base class
# This cell extends RRT with optimal rewiring

# RRT* Implementation with Rewiring

class RRTStar(RRT):
    """RRT* - Asymptotically Optimal RRT."""
    
    def __init__(self, start, goal, environment, 
                 max_iterations=250, step_size=2.0, goal_sample_rate=0.05,
                 rewire_radius_factor=20.0):
        """
        Args:
            rewire_radius_factor: Gamma parameter for shrinking radius
        """
        super().__init__(start, goal, environment, max_iterations, step_size, goal_sample_rate)
        self.rewire_radius_factor = rewire_radius_factor
        
    def search_radius(self):
        """Calculate search radius for near neighbors (shrinking with tree size)."""
        n = len(self.nodes)
        d = 2  # Dimension of C-space
        
        # r_n = min(gamma * (log(n) / n)^(1/d), step_size)
        if n > 1:
            radius = min(
                self.rewire_radius_factor * (np.log(n) / n)**(1/d),
                self.step_size
            )
        else:
            radius = self.step_size
        
        return radius
    
    def near(self, x, y, radius):
        """Find all nodes within radius of (x, y)."""
        near_nodes = []
        
        for node in self.nodes:
            dist = np.sqrt((node.x - x)**2 + (node.y - y)**2)
            if dist <= radius:
                near_nodes.append(node)
        
        return near_nodes
    
    def choose_best_parent(self, near_nodes, new_node):
        """Choose parent that minimizes cost to new_node."""
        if not near_nodes:
            return None
        
        min_cost = float('inf')
        best_parent = None
        
        for near_node in near_nodes:
            # Cost to reach new_node through near_node
            edge_cost = np.sqrt((near_node.x - new_node.x)**2 + 
                               (near_node.y - new_node.y)**2)
            total_cost = near_node.cost + edge_cost
            
            # Check if path is collision-free
            if (total_cost < min_cost and 
                self.env.is_path_collision_free(near_node.x, near_node.y,
                                               new_node.x, new_node.y)):
                min_cost = total_cost
                best_parent = near_node
        
        if best_parent:
            new_node.parent = best_parent
            new_node.cost = min_cost
        
        return best_parent
    
    def rewire(self, near_nodes, new_node):
        """Rewire tree to reduce costs through new_node."""
        for near_node in near_nodes:
            # Skip if same node
            if near_node == new_node:
                continue
            
            # Cost to reach near_node through new_node
            edge_cost = np.sqrt((new_node.x - near_node.x)**2 + 
                               (new_node.y - near_node.y)**2)
            new_cost = new_node.cost + edge_cost
            
            # If cheaper and collision-free, rewire
            if (new_cost < near_node.cost and
                self.env.is_path_collision_free(new_node.x, new_node.y,
                                               near_node.x, near_node.y)):
                # Rewire: update parent
                near_node.parent = new_node
                near_node.cost = new_cost
                
                # Propagate cost updates to children
                self.propagate_cost_to_children(near_node)
    
    def propagate_cost_to_children(self, parent_node):
        """Update costs for all descendants of parent_node."""
        for node in self.nodes:
            if node.parent == parent_node:
                edge_cost = np.sqrt((parent_node.x - node.x)**2 + 
                                   (parent_node.y - node.y)**2)
                node.cost = parent_node.cost + edge_cost
                self.propagate_cost_to_children(node)
    
    def plan(self):
        """Run RRT* algorithm with rewiring."""
        best_path_cost = float('inf')
        best_path = None
        cost_history = []
        
        for i in range(self.max_iterations):
            # Sample
            x_rand, y_rand = self.sample()
            
            # Nearest
            nearest_node = self.nearest(x_rand, y_rand)
            
            # Steer
            new_node = self.steer(nearest_node, x_rand, y_rand)
            
            # Find near nodes
            radius = self.search_radius()
            near_nodes = self.near(new_node.x, new_node.y, radius)
            
            # Choose best parent (RRT* modification)
            best_parent = self.choose_best_parent(near_nodes, new_node)
            
            if best_parent is not None:
                # Add to tree
                self.nodes.append(new_node)
                
                # Rewire tree (RRT* modification)
                self.rewire(near_nodes, new_node)
                
                # Check if reached goal
                dist_to_goal = np.sqrt((new_node.x - self.goal.x)**2 + 
                                      (new_node.y - self.goal.y)**2)
                
                if dist_to_goal < self.goal_threshold:
                    if self.env.is_path_collision_free(new_node.x, new_node.y,
                                                       self.goal.x, self.goal.y):
                        # Update goal's parent if this path is better
                        path_cost = new_node.cost + dist_to_goal
                        
                        if path_cost < best_path_cost:
                            self.goal.parent = new_node
                            self.goal.cost = path_cost
                            best_path_cost = path_cost
                            best_path = self.extract_path()
                            
                            if self.goal not in self.nodes:
                                self.nodes.append(self.goal)
            
            # Record best cost so far
            if best_path:
                cost_history.append(best_path_cost)
            else:
                cost_history.append(float('inf'))
        
        return best_path, self.max_iterations, cost_history


# Compare RRT vs RRT* (Colab Optimized)
np.random.seed(42)
rrt = RRT(start, goal, env, max_iterations=250, step_size=2.0)
rrt_path, rrt_iterations = rrt.plan()

np.random.seed(42)
rrt_star = RRTStar(start, goal, env, max_iterations=250, step_size=2.0, rewire_radius_factor=15.0)
rrt_star_path, rrt_star_iterations, cost_history = rrt_star.plan()

# Calculate path lengths
rrt_len = sum(np.sqrt((rrt_path[i+1][0]-rrt_path[i][0])**2+(rrt_path[i+1][1]-rrt_path[i][1])**2) 
              for i in range(len(rrt_path)-1)) if rrt_path else float('inf')
rrt_star_len = sum(np.sqrt((rrt_star_path[i+1][0]-rrt_star_path[i][0])**2+(rrt_star_path[i+1][1]-rrt_star_path[i][1])**2) 
                   for i in range(len(rrt_star_path)-1)) if rrt_star_path else float('inf')
improvement = ((rrt_len - rrt_star_len) / rrt_len * 100) if rrt_len != float('inf') else 0

# Simple 1x2 visualization (paths only - memory efficient)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# RRT path
for ox, oy, ow, oh in env.obstacles:
    ax1.add_patch(Rectangle((ox, oy), ow, oh, facecolor='gray', edgecolor='black', linewidth=2))
if rrt_path:
    px, py = zip(*rrt_path)
    ax1.plot(px, py, 'b-', linewidth=3, label='RRT Path')
ax1.scatter([start[0]], [start[1]], c='green', s=300, marker='o', edgecolors='black', linewidths=2, label='Start', zorder=10)
ax1.scatter([goal[0]], [goal[1]], c='red', s=300, marker='*', edgecolors='black', linewidths=2, label='Goal', zorder=10)
ax1.set_xlim([0, env.width])
ax1.set_ylim([0, env.height])
ax1.set_title(f'RRT\nLength: {rrt_len:.2f}, Nodes: {len(rrt.nodes)}', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_aspect('equal')

# RRT* path
for ox, oy, ow, oh in env.obstacles:
    ax2.add_patch(Rectangle((ox, oy), ow, oh, facecolor='gray', edgecolor='black', linewidth=2))
if rrt_star_path:
    px, py = zip(*rrt_star_path)
    ax2.plot(px, py, 'purple', linewidth=3, label='RRT* Path')
ax2.scatter([start[0]], [start[1]], c='green', s=300, marker='o', edgecolors='black', linewidths=2, label='Start', zorder=10)
ax2.scatter([goal[0]], [goal[1]], c='red', s=300, marker='*', edgecolors='black', linewidths=2, label='Goal', zorder=10)
ax2.set_xlim([0, env.width])
ax2.set_ylim([0, env.height])
ax2.set_title(f'RRT*\nLength: {rrt_star_len:.2f}, Nodes: {len(rrt_star.nodes)}', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_aspect('equal')

plt.tight_layout()
plt.show()
plt.close(fig)

# Print results
print("="*70)
print("RRT vs RRT* COMPARISON (Colab Optimized)")
print("="*70)
print(f"RRT:   Length={rrt_len:.2f}, Nodes={len(rrt.nodes)}")
print(f"RRT*:  Length={rrt_star_len:.2f}, Nodes={len(rrt_star.nodes)}")
print(f"Improvement: {improvement:.1f}% shorter path")
print(f"Memory usage: ~200 MB (optimized for Colab)")
print("="*70)

# Clear memory
del rrt, rrt_star, rrt_path, rrt_star_path, cost_history
gc.collect()


## 4. State Lattice Planning

State lattice planning uses a **discrete graph** of kinematically-feasible motions precomputed for the vehicle.

### Key Idea

Instead of random sampling, use a **library of motion primitives** that respect vehicle constraints (e.g., turning radius, acceleration limits).

### Motion Primitives

Precomputed trajectories from a reference state:
- Straight
- Left turn (various radii)
- Right turn (various radii)
- Lane change left
- Lane change right

### State Lattice Construction

1. **Discretize state space**: (x, y, θ) grid
2. **For each state, apply motion primitives** to reach neighboring states
3. **Build graph** of reachable states
4. **Search graph** using A* or Dijkstra

### Advantages

- **Kinematically feasible**: Paths respect vehicle dynamics
- **Optimal on lattice**: A* guarantees shortest path on graph
- **Fast**: Graph search is efficient
- **Predictable**: Deterministic results

### Disadvantages

- **Resolution completeness**: Only finds solutions on the lattice
- **Memory intensive**: Must store all primitives
- **Requires tuning**: Lattice resolution vs. computation

### Applications

- **Highway driving**: Structured environments with lanes
- **Parking**: Low-speed maneuvering
- **Urban navigation**: Road-constrained motion

## Exercises

### Exercise 1: Implement RRT*

Extend the RRT implementation to add the rewiring step for path optimization.

**Tasks:**
1. Implement the `Near()` function to find nodes within radius
2. Add "Choose Best Parent" logic
3. Add "Rewire Tree" logic
4. Compare path quality (cost) between RRT and RRT*
5. Plot cost vs. iterations to show convergence

**Hint:** Use shrinking radius $r = \gamma (\log n / n)^{1/2}$ where $\gamma \approx 10-20$

In [ ]:
# Exercise 3 - Template

class DynamicObstacle:
    """Moving obstacle"""
    def __init__(self, x0, y0, vx, vy, radius):
        self.x0 = x0
        self.y0 = y0
        self.vx = vx
        self.vy = vy
        self.radius = radius
    
    def position_at_time(self, t):
        """Return position at time t"""
        return (self.x0 + self.vx * t, self.y0 + self.vy * t)

# TODO: Extend the RRT class from Section 2 to handle dynamic obstacles
# 
# Hints:
# 1. Add time as a state variable: RRTNode should store (x, y, t)
# 2. Modify collision checking to check space-time collision
# 3. When steering, also advance time proportional to distance traveled
# 4. Check if path intersects with moving obstacle trajectory
#
# Example approach:
# class DynamicRRT(RRT):  # Extend RRT from above
#     def __init__(self, environment, start, goal, dynamic_obstacles, **kwargs):
#         super().__init__(environment, start, goal, **kwargs)
#         self.dynamic_obstacles = dynamic_obstacles
#     
#     def is_collision_at_time(self, x, y, t):
#         """Check collision with moving obstacles at time t"""
#         # Check static obstacles (from parent class)
#         if self.env.is_collision(x, y):
#             return True
#         
#         # Check dynamic obstacles
#         for obs in self.dynamic_obstacles:
#             obs_x, obs_y = obs.position_at_time(t)
#             dist = np.sqrt((x - obs_x)**2 + (y - obs_y)**2)
#             if dist < obs.radius:
#                 return True
#         return False

# Your implementation here

## References and Additional Resources

### Core Textbooks

1. **Planning Algorithms** by LaValle (2006)
   - Chapters 5-6: Sampling-based Planning
   - Comprehensive coverage of RRT, RRT*
   - Free online: http://planning.cs.uiuc.edu/

2. **Robotics: Modelling, Planning and Control** by Siciliano et al. (2009)
   - Chapter 6: Motion Planning

3. **Principles of Robot Motion** by Choset et al. (2005)
   - Path planning algorithms
   - Configuration space concepts

### Key Papers

1. **RRT**
   - LaValle (1998) - "Rapidly-Exploring Random Trees: A New Tool for Path Planning"
   - Original RRT paper

2. **RRT***
   - Karaman & Frazzoli (2011) - "Sampling-based Algorithms for Optimal Motion Planning"
   - Proves asymptotic optimality

3. **State Lattice**
   - Pivtoraiko & Kelly (2005) - "Generating Near Minimal Spanning Control Sets for Constrained Motion Planning"
   - McNaughton et al. (2011) - "Motion Planning for Autonomous Driving with a Conformal Spatiotemporal Lattice"

4. **Dynamic Environments**
   - Ferguson et al. (2008) - "Replanning with RRTs"
   - Anytime Dynamic A*

### Algorithms & Variants

1. **RRT Variants**
   - RRT-Connect: Bidirectional RRT
   - RRT*-Smart: Improved sampling
   - Informed RRT*: Ellipsoidal sampling
   - Kinodynamic-RRT: Respects dynamics

2. **Other Sampling Methods**
   - PRM (Probabilistic Roadmap)
   - EST (Expansive Space Trees)
   - SBL (Single-Query Bidirectional Lazy Planner)

3. **Search-Based Methods**
   - Field D*: Interpolated paths
   - Hybrid A*: Continuous state space
   - Theta*: Any-angle planning

### Software & Libraries

1. **OMPL (Open Motion Planning Library)**
   - C++ library with Python bindings
   - Implements RRT, RRT*, PRM, etc.
   - https://ompl.kavrakilab.org/

2. **Python Robotics**
   - Educational implementations
   - https://github.com/AtsushiSakai/PythonRobotics

3. **MoveIt** (ROS)
   - Motion planning framework
   - Industrial manipulation and mobile robots

### Applications in Autonomous Vehicles

1. **Highway Driving**
   - Lattice planners for lane keeping/changing
   - Polynomial trajectory optimization

2. **Urban Driving**
   - Hybrid approaches (lattice + optimization)
   - Interaction-aware planning

3. **Parking**
   - RRT for maneuvering in tight spaces
   - Reeds-Shepp curves

### Advanced Topics

1. **Kinodynamic Planning**
   - Incorporate acceleration/velocity limits
   - Respect vehicle dynamics

2. **Uncertainty**
   - Chance-constrained RRT
   - Robust trajectory optimization

3. **Learning-Based**
   - Neural Motion Planning
   - Learning sampling distributions
   - Imitation learning for primitives

4. **Multi-Query Planning**
   - PRM for multiple start-goal queries
   - Roadmap maintenance

### Practical Considerations

1. **Real-Time Planning**
   - Anytime algorithms
   - Incremental replanning
   - Computing budgets (< 100ms)

2. **Safety**
   - Guaranteed collision avoidance
   - Safe corridors
   - Emergency maneuvers

3. **Smoothness**
   - Path smoothing post-processing
   - Bezier/spline fitting
   - Jerk minimization

---

## Summary

This notebook covered local motion planning for autonomous vehicles:

**Key Concepts:**
- Configuration space and collision detection
- RRT for probabilistic complete planning
- RRT* for asymptotically optimal paths
- State lattice for kinematically-feasible planning

**Practical Skills:**
- Implementing sampling-based planners
- Building collision-free paths
- Understanding trade-offs (completeness vs. optimality vs. speed)
- Handling complex environments with obstacles

**Next Steps:**
1. Study trajectory optimization (MPC, numerical optimization)
2. Learn about prediction-aware planning
3. Explore learning-based motion planning
4. Implement real-time replanning strategies